# Voxels data - what are we actually visualizing in the histogram?

I wanted to see exactly what the values in my histogram represented, and whether I had intepreted them correctly. Especially when I had put so much effort into making sure the colormaps corresponded with the fMRI images. This is done by tying the voxel indices to an already established "brain coordinates map", such as the MNI coordinates which can be accessed through panda. 

**Below are 3 code sections, depending on how much data you might want to visualize:**
1. The data in its entirety, with all voxels (except "empty" values).
2. The voxel indices and corresponding coordinates for the uniformity data.
3. The voxel indices and corresponding coordinates for the association data.

First, we need to get the voxel data ready for analysis. This includes loading the voxel data (again) with the neccessary packages, and converting the voxel data into a more name friendly format for this specific task. The major change of loading the NIfTI files as img_ instead of functional_

In [3]:

import os
import nibabel as nib
import numpy as np

folder_path = '/Users/sirikagen/Desktop/Bipolar-fMRI/fMRI files/157 studies'  # Adjust to your folder
files = os.listdir(folder_path)

functional_file = next(f for f in files if "uniformity" in f and f.endswith('uniformity_157.nii'))
functional_file2 = next(f for f in files if "association" in f and f.endswith('association_157.nii'))

# Load the NIfTI files
img_uniformity= nib.load(os.path.join(folder_path, functional_file))
img_association = nib.load(os.path.join(folder_path, functional_file2))


# Get data array
uniformity = img_uniformity.get_fdata()
association = img_uniformity.get_fdata()

## Alternative 1 - The Entirety Of The Data

In [5]:
import pandas as pd

# Get the voxel data arrays
uniformity_data = img_uniformity.get_fdata()
association_data = img_association.get_fdata()

# Get affine matrix for voxel-to-MNI conversion
affine = img_uniformity.affine

# Find non-zero voxel indices and their values
nonzero_indices = np.array(np.nonzero(uniformity_data)).T  # Indices of non-zero voxels
nonzero_uniformity_values = uniformity_data[nonzero_indices[:, 0], 
                                            nonzero_indices[:, 1], 
                                            nonzero_indices[:, 2]]

# Map voxel indices to MNI coordinates - ties our voxel data to an already established "map" for fMRI coordinates in the brain
mni_coords = nib.affines.apply_affine(affine, nonzero_indices)

# Create a DataFrame to store the results
df = pd.DataFrame({
    'Voxel_X': mni_coords[:, 0],
    'Voxel_Y': mni_coords[:, 1],
    'Voxel_Z': mni_coords[:, 2],
    'Uniformity_Value': nonzero_uniformity_values
})

# Add association values to the DataFrame
nonzero_association_values = association_data[nonzero_indices[:, 0], 
                                              nonzero_indices[:, 1], 
                                              nonzero_indices[:, 2]]
df['Association_Value'] = nonzero_association_values

#Display the first few rows of the table (nice little sneak peak that your code is constructing the table like intended)
print(df.head())

# Save the voxel values (ALL voxel values from the original data files) to a CSV file
df.to_csv('voxel__full_data.csv', index=False)

   Voxel_X  Voxel_Y  Voxel_Z  Uniformity_Value  Association_Value
0     66.0    -50.0     16.0          3.653485                0.0
1     66.0    -48.0     12.0          3.653485                0.0
2     66.0    -48.0     14.0          3.653485                0.0
3     66.0    -48.0     16.0          3.653485                0.0
4     66.0    -46.0     14.0          3.653485                0.0


## Alternative 2 - The Corresponding Coordinates Of The "Uniformity Values"

In [6]:

# Get the voxel data arrays
uniformity_data = img_uniformity.get_fdata()

# Filter the data array to include only values within the specified range

# Get affine matrix for voxel-to-MNI conversion
affine = img_uniformity.affine

# Get the shape of the voxel array
voxel_shape = uniformity_data.shape

# Generate all possible voxel indices
voxel_indices = np.array(np.meshgrid(
    np.arange(voxel_shape[0]),
    np.arange(voxel_shape[1]),
    np.arange(voxel_shape[2])
)).reshape(3, -1).T  # Create a full grid of (x, y, z) indices

# Map voxel indices to MNI coordinates - ties our voxel data to an already established "map" for fMRI coordinates in the brain
mni_coords = nib.affines.apply_affine(affine, voxel_indices)


# Extract intensity values for uniformity and association 
uniformity_values = uniformity_data[voxel_indices[:, 0], 
                                    voxel_indices[:, 1], 
                                    voxel_indices[:, 2]]


# Create a DataFrame to store the results
df = pd.DataFrame({
    'Voxel_X': mni_coords[:, 0],
    'Voxel_Y': mni_coords[:, 1],
    'Voxel_Z': mni_coords[:, 2],
    'Uniformity_Value': uniformity_values
})

#Filter only the positive values, to avoid having to scroll through unrelevant 0.0 data (it is a lot)
df_filtered = df[(df['Uniformity_Value'] > 0)]

#Display the first few rows of the table (nice little sneak peak that your code is constructing the table like intended)
df_sorted = df_filtered.sort_values(by='Uniformity_Value').reset_index(drop=True)
display(df_sorted)

# Save the voxel values (conviniently the ones visualized in the "uniformity" histogram) to a CSV file
df_sorted.to_csv('voxel_uniformity_histogram.csv', index=False)

,Voxel_X,Voxel_Y,Voxel_Z,Uniformity_Value
0,-30.0,-92.0,0.0,3.653485
1,50.0,18.0,16.0,3.653485
2,50.0,18.0,26.0,3.653485
3,50.0,18.0,28.0,3.653485
4,48.0,18.0,-2.0,3.653485
...,...,...,...,...
7465,32.0,24.0,-4.0,13.437421
7466,34.0,22.0,0.0,14.252749
7467,34.0,20.0,-4.0,14.252749
7468,34.0,20.0,-2.0,14.252749


## Alternative 3 - The Corresponding Coordinates Of The "Association Values"

In [7]:
import pandas as pd
import numpy as np
import nibabel as nib

# Get the voxel data arrays
association_data = img_association.get_fdata()

# Get affine matrix for voxel-to-MNI conversion
affine = img_association.affine

# Get the shape of the voxel array
voxel_shape = association_data.shape

# Generate all possible voxel indices
voxel_indices = np.array(np.meshgrid(
    np.arange(voxel_shape[0]),
    np.arange(voxel_shape[1]),
    np.arange(voxel_shape[2])
)).reshape(3, -1).T  # Create a full grid of (x, y, z) indices

# Map voxel indices to MNI coordinates - ties our voxel data to an already established "map" for fMRI coordinates in the brain
mni_coords = nib.affines.apply_affine(affine, voxel_indices)

# Extract intensity values for uniformity and association
association_values = association_data[voxel_indices[:, 0], 
                                      voxel_indices[:, 1], 
                                      voxel_indices[:, 2]]

# Create a DataFrame to store the results
df = pd.DataFrame({
    'Voxel_X': mni_coords[:, 0],
    'Voxel_Y': mni_coords[:, 1],
    'Voxel_Z': mni_coords[:, 2],
    'Association_Value': association_values
})


#Filter only the positive values, to avoid having to scroll through unrelevant 0.0 data 
df_filtered = df[(df['Association_Value'] > 0)]

#Display the first few rows of the table (nice little sneak peak that your code is constructing the table like intended)
df_sorted = df_filtered.sort_values(by='Association_Value').reset_index(drop=True)
display(df_sorted)

# Save the voxel values (conviniently the ones visualized in the "association" histogram) to a CSV file
df_sorted.to_csv('voxel_association_histogram.csv', index=False)


,Voxel_X,Voxel_Y,Voxel_Z,Association_Value
0,-6.0,-46.0,-4.0,4.509738
1,-38.0,22.0,-38.0,4.512328
2,-18.0,14.0,-14.0,4.516972
3,2.0,16.0,-12.0,4.516972
4,24.0,44.0,6.0,4.530419
...,...,...,...,...
145,-6.0,-8.0,-38.0,6.654262
146,-10.0,-10.0,-38.0,6.654262
147,-10.0,4.0,-36.0,6.654262
148,-8.0,-8.0,-38.0,9.515321


## You got the coordinates - how to interpret them?

Please see attached image "Explaination_Association values" for å visualization of what the voxel data represents, and how it correlates to the data we pull from NeuroSynth: [https://neurosynth.org/analyses/terms/bipolar/]